### 0. 토지임야정보 데이터 설명

대전광역시 행정구역코드(법정동코드) : 3000000000 ~ 3100000000 사이의 코드

- 항목명
1. pnu : 고유번호
2. IdCode : 법정동코드
3. IdCodeNm : 법정동명
4. mnnmSlno : 지번
5. lndcgrCode : 지목코드 - 토지의 주된 용도에 따라 토지의 종류를 구분한 지목코드

    01: 전  02: 답  03: 과수원  04: 목장용지  05: 임야  06: 광천지
   
    07: 염전  08: 대  09: 공장용지  10: 학교용지  11: 주차장
   
    12: 주요소용지  13: 창고용지  14: 도로  15: 철도용지  16: 제방
   
    17: 하천  18: 구거  19: 유지  20: 양어장  21: 수도용지  22: 공원
    
    23: 체육용지  24: 유원지  25: 종교용지  26: 사적지  27: 묘지
    
    28: 잡종지
    
6. IndcgrCodeNm : 지목명 

7. posesnSeCode : 소유구분코드 - 국토를 토지 소유권 취득 주체에 따라 구분한 코드
    00 : 일본인, 창씨명 등  01 : 개인  02 : 국유지
    
    03 : 외국인, 외국공공기관  04 : 도유  05 : 군유지  06 : 법인
    
    07 : 종중  08 : 종교단체  09 : 기타단체
    
8. posesnSeCodeNm : 소유구분명 (ex. 법인)

9. regstrSeCode : 대장구분코드 (ex.1 : 일반, 2 : 산)

### 1. 필요 라이브러리 

In [4]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [33]:
# 대전 법정동 코드
daejeon_num = [3011010100,
3011010200,
3011010300,
3011010400,
3011010500,
3011010600,
3011010700,
3011010800,
3011010900,
3011011000,
3011011100,
3011011200,
3011011300,
3011011400,
3011011500,
3011011600,
3011011700,
3011011800,
3011011900,
3011012000,
3011012100,
3011012200,
3011012300,
3011012400,
3011012500,
3011012600,
3011012700,
3011012800,
3011012900,
3011013000,
3011013100,
3011013200,
3011013300,
3011013400,
3011013500,
3011013600,
3011013700,
3011013800,
3011013900,
3011014000,
3011014100,
3011014200,
3011014300,
3011014400,
3011014500,
3014010100,
3014010200,
3014010300,
3014010400,
3014010500,
3014010600,
3014010700,
3014010800,
3014010900,
3014011000,
3014011100,
3014011200,
3014011300,
3014011400,
3014011500,
3014011600,
3014011700,
3014011800,
3014011900,
3014012000,
3014012100,
3014012200,
3014012300,
3014012400,
3014012500,
3014012600,
3017010100,
3017010200,
3017010300,
3017010400,
3017010500,
3017010600,
3017010700,
3017010800,
3017010900,
3017011000,
3017011100,
3017011200,
3017011300,
3017011400,
3017011500,
3017011600,
3017011700,
3017011800,
3017011900,
3017012000,
3017012100,
3017012200,
3017012300,
3017012400,
3017012500,
3017012600,
3017012700,
3017012800,
3020010100,
3020010200,
3020010300,
3020010400,
3020010500,
3020010600,
3020010700,
3020010800,
3020010900,
3020011000,
3020011100,
3020011200,
3020011300,
3020011400,
3020011500,
3020011600,
3020011700,
3020011800,
3020011900,
3020012000,
3020012100,
3020012200,
3020012300,
3020012400,
3020012500,
3020012600,
3020012700,
3020012800,
3020012900,
3020013000,
3020013100,
3020013200,
3020013300,
3020013400,
3020013500,
3020013600,
3020013700,
3020013800,
3020013900,
3020014000,
3020014100,
3020014200,
3020014300,
3020014400,
3020014500,
3020014600,
3020014700,
3020014800,
3020014900,
3020015000,
3020015100,
3020015200,
3020015300,
3023010100,
3023010200,
3023010300,
3023010400,
3023010500,
3023010600,
3023010700,
3023010800,
3023010900,
3023011000,
3023011100,
3023011200,
3023011300,
3023011400,
3023011500,
3023011600,
3023011700,
3023011800,
3023011900,
3023012000,
3023012100,
3023012200,
3023012300,
3023012400,
3023012500,
3023012600]

### 2. Open API 데이터 불러오기

In [35]:
# 1. URL 파라미터 분리하기
# Service URL
xmlUrl = "http://apis.data.go.kr/1611000/nsdi/eios/LadfrlService/ladfrlList.xml"
My_API_Key = unquote("DZmq1Zbk1ohdHMUAi%2BysDvzhfimwdCa5S7Zkrstme%2FxcK33DZYmjRugvRb9JtpMh9Ef%2F1b%2FyYrm1ftXE891wIw%3D%3D")

In [63]:
# 대전 동별 행정동코드로 데이터 불러오기
for pnu in daejeon_num:
    queryParams = '?' + urlencode(   # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다.
            {
                quote_plus('ServiceKey') : My_API_Key, # 필수 항목 1 : 서비스키(본인의 서비스키)
                quote_plus('pnu') : pnu
            }
    )
    response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    rows = xmlobj.findAll('ladfrlVOList')
    
    rowList = []
    nameList = []
    columnList = []
    rowsLen = len(rows)
    for i in range(0, rowsLen):
        columns = rows[i].find_all()
    
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장
            if i == 0:
                nameList.append(columns[j].name)
            # 컬럼값은 모든 행의 값을 저장해야한다.
            eachColumn = columns[j].text
            columnList.append(eachColumn)
            
        rowList.append(columnList)
        columnList = [] # 다음 row의 값을 넣기 위해 비워준다,
    
    if pnu == 3011010100: # 처음 데이터 프레임을 만들 때
        result_base = pd.DataFrame(rowList,columns = nameList)
    else:
        result_ = pd.DataFrame(rowList,columns = nameList)
        result_base = pd.concat([result_base,result_], ignore_index = True)

### 3. 결과

In [65]:
result_base

,pnu,ldCode,ldCodeNm,mnnmSlno,regstrSeCode,regstrSeCodeNm,lndcgrCode,lndcgrCodeNm,lndpclAr,posesnSeCode,posesnSeCodeNm,cnrsPsnCo,ladFrtlSc,ladFrtlScNm,lastUpdtDt
0,3011010100100200001,3011010100,대전광역시 동구 원동,20-1,1,토지대장,08,대,165,01,개인,0,06,1:600,2020-05-20
1,3011010100100200002,3011010100,대전광역시 동구 원동,20-2,1,토지대장,08,대,32.1,01,개인,1,06,1:600,2020-05-20
2,3011010100100200003,3011010100,대전광역시 동구 원동,20-3,1,토지대장,08,대,329.3,01,개인,2,06,1:600,2020-06-19
3,3011010100100200005,3011010100,대전광역시 동구 원동,20-5,1,토지대장,08,대,189.8,01,개인,0,06,1:600,2020-05-20
4,3011010100100200006,3011010100,대전광역시 동구 원동,20-6,1,토지대장,08,대,99.2,01,개인,0,06,1:600,2020-05-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,3023012600100030000,3023012600,대전광역시 대덕구 신탄진동,3-0,1,토지대장,08,대,341,02,국유지,0,12,1:1200,2020-05-20
1766,3023012600100040001,3023012600,대전광역시 대덕구 신탄진동,4-1,1,토지대장,01,전,1117,02,국유지,0,12,1:1200,2020-05-20
1767,3023012600100040002,3023012600,대전광역시 대덕구 신탄진동,4-2,1,토지대장,21,수도용지,3669,02,국유지,0,12,1:1200,2020-05-20
1768,3023012600100040003,3023012600,대전광역시 대덕구 신탄진동,4-3,1,토지대장,01,전,423,02,국유지,0,12,1:1200,2020-05-20


### 4. 엑셀로 저장

In [66]:
# 엑셀로 저장
result_base.to_excel('대전토지임야정보.xlsx',index=False)